<a href="https://colab.research.google.com/github/naga-dheeraj-p/Assignments/blob/main/_Consolidation_Breakout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocessing

In [ ]:
pip install fyers_apiv3

In [ ]:
import pandas as pd
df = pd.read_csv("/content/2000_STOCKS_list.csv")
df = df[df[" SERIES"].str.strip() == "EQ"]
symbols = df["SYMBOL"].str.strip().tolist()
df = pd.read_csv("/content/stocks_above_6kcr.csv")
df = df[df[" SERIES"].str.strip() == "EQ"]
symbols = df["SYMBOL"].str.strip().tolist()
symbols
fyers_symbols_filtered = [f"NSE:{sym}-EQ" for sym in symbols]
print(fyers_symbols_filtered[:10])

In [ ]:
df = pd.read_csv("/content/stocks_above_6kcr.csv")
df = df[df[" SERIES"].str.strip() == "EQ"]
symbols = df["SYMBOL"].str.strip().tolist()
symbols
fyers_symbols_filtered = [f"NSE:{sym}-EQ" for sym in symbols]
print(fyers_symbols_filtered[:10])

# Dates

In [ ]:
import pandas as pd
df = pd.read_csv('/content/stocks_for_date.csv')
df['Date'] = pd.to_datetime(df['Date'])
date_list = df['Date'].dt.strftime('%Y-%m-%d').tolist()
print(date_list)

In [ ]:
date_list=date_list[43:]
date_list

In [ ]:
import pandas as pd
df = pd.read_csv("/content/may_to_aug.csv", header=0)
first_col = df.iloc[:, 0]
converted_dates = pd.to_datetime(first_col, format='%d-%b-%Y').dt.strftime('%Y-%m-%d').tolist()
print(converted_dates)

In [ ]:
converted_dates=converted_dates[-21:]
converted_dates

# Strategy

In [ ]:
# 1% range and 0.75% candle
#***************************************************************************************************************************
import pandas as pd
import numpy as np
import time
from fyers_apiv3 import fyersModel
from collections import defaultdict

# === Fyers Setup ===
client_id = "6HK249XFBQ-100"
access_token = "eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJhdWQiOlsiZDoxIiwiZDoyIiwieDowIiwieDoxIiwieDoyIl0sImF0X2hhc2giOiJnQUFBQUFCb2R3MnpXYU5jV0FFVEVVa09pMEd2TVRkcWxVcEFYOW9SQkdXQ3ZLV0Z4Z3FsMVVpekx5T2JicnMtLXFDMVNadnFiOGYzQ3d0WDV5eVhHd3pCdTRJb3BoR2k2bzc2TlM5NXgzbExSYjdxbWxOcE1fbz0iLCJkaXNwbGF5X25hbWUiOiIiLCJvbXMiOiJLMSIsImhzbV9rZXkiOiIxNGUwYzFlNGYzZTU0MDUzNGI4YzJkMDA1YjdlMTU0ODg0M2NhMzJkYmFhZjc2ZjQzMDQ5YzdlMCIsImlzRGRwaUVuYWJsZWQiOiJOIiwiaXNNdGZFbmFibGVkIjoiTiIsImZ5X2lkIjoiRkFBODM3MDUiLCJhcHBUeXBlIjoxMDAsImV4cCI6MTc1MjcxMjIwMCwiaWF0IjoxNzUyNjMyNzU1LCJpc3MiOiJhcGkuZnllcnMuaW4iLCJuYmYiOjE3NTI2MzI3NTUsInN1YiI6ImFjY2Vzc190b2tlbiJ9.aob4QJtXf40RvMwUb53cRWCJNGWKFymKrIXa8SszzIQ"
fyers = fyersModel.FyersModel(client_id=client_id, token=access_token)

# === Load Market Cap Data ===
market_cap_df = pd.read_csv('/content/stocks_above_1kcr.csv')
market_cap_dict = dict(zip(market_cap_df['SYMBOL'], market_cap_df['MARKET_CAP_CR']))

# === Parameters ===
window_size = 24
max_range_pct = 0.01
volume_multiplier = 5
entry_buffer_pct = 0.01
min_body_pct = 0.0075
confirmation_breakout_pct = 0.01

# === Fetch 5-min Candle Data ===
def fetch_5min_data(symbol, date):
    try:
        res = fyers.history({
            "symbol": symbol,
            "resolution": "5",
            "date_format": "1",
            "range_from": date,
            "range_to": date,
            "cont_flag": "0"
        })
        if 'candles' in res and len(res['candles']) > 0:
            df = pd.DataFrame(res['candles'], columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s', utc=True).dt.tz_convert('Asia/Kolkata')
            return df
        else:
            return pd.DataFrame()
    except Exception as e:
        return pd.DataFrame()

# === Simulate Exit ===
def simulate_exit(entry_price, sl, candles_after_entry):
    for _, row in candles_after_entry.iterrows():
        if row['low'] <= sl:
            return row['timestamp'], sl, ((sl - entry_price) / entry_price) * 100, 'SL Hit'
    final_price = candles_after_entry.iloc[-1]['close']
    return candles_after_entry.iloc[-1]['timestamp'], final_price, ((final_price - entry_price) / entry_price) * 100, 'End of Day Exit'

# === Strategy ===
def dynamic_range_breakout(df, symbol, date):
    trades = []
    if df.empty or len(df) < window_size + 10:
        return trades

    for i in range(len(df) - window_size - 6):
        range_df = df.iloc[i:i + window_size]
        r_high = range_df['high'].max()
        r_low = range_df['low'].min()
        r_width = (r_high - r_low) / r_low
        mean_volume = range_df['volume'].mean()

        if r_width < max_range_pct:
            breakout_candle = df.iloc[i + window_size]

            if breakout_candle['timestamp'].time() > pd.to_datetime("14:30").time():
                continue

            body = breakout_candle['close'] - breakout_candle['open']
            upper_wick = breakout_candle['high'] - breakout_candle['close']
            body_pct = body / breakout_candle['open']

            if upper_wick > 0.4 * abs(body):
                continue

            if (
                breakout_candle['close'] > breakout_candle['open'] and
                breakout_candle['high'] > r_high and
                body_pct >= min_body_pct and
                breakout_candle['volume'] > volume_multiplier * mean_volume
            ):
                target_entry_price = r_high * (1 + confirmation_breakout_pct)
                candles_after = df.iloc[i + window_size + 1:]

                for j in range(len(candles_after)):
                    row = candles_after.iloc[j]
                    if row['timestamp'].time() > pd.to_datetime("14:30").time():
                        break
                    if row['high'] >= target_entry_price:
                        entry_price = target_entry_price
                        sl = entry_price * 0.99
                        remaining_candles = candles_after.iloc[j + 1:]
                        if len(remaining_candles) == 0:
                            break
                        exit_time, exit_price, pnl_pct, reason = simulate_exit(entry_price, sl, remaining_candles)
                        symbol_key = symbol.split(':')[-1].split('-')[0]
                        market_cap = market_cap_dict.get(symbol_key, np.nan)
                        trades.append({
                            'date': date,
                            'symbol': symbol,
                            'entry_time': row['timestamp'],
                            'entry_price': round(entry_price, 2),
                            'exit_time': exit_time,
                            'exit_price': round(exit_price, 2),
                            'pnl_%': round(pnl_pct, 2),
                            'reason': reason
                        })
                        return trades
    return trades

# === Inputs ===
dates_to_check = converted_dates
fyers_symbols = fyers_symbols_filtered  # List of symbols

# === Portfolio Summary Setup ===
portfolio_value = 100000
date_to_returns = {}
date_to_trades = {}

monthly_returns = defaultdict(float)
monthly_trades = defaultdict(int)

# === Run Strategy ===
for date in dates_to_check:
    daily_trades = []
    for symbol in fyers_symbols:
        df = fetch_5min_data(symbol, date)
        if not df.empty:
            trades = dynamic_range_breakout(df, symbol, date)
            daily_trades.extend(trades)
        time.sleep(0)  # Avoid rate limiting

    if daily_trades:
        daily_df = pd.DataFrame(daily_trades)
        pnls = daily_df['pnl_%'].values
        day_return = round(np.sum(pnls), 2)
        num_trades = len(pnls)
    else:
        day_return = 0
        num_trades = 0

    # Portfolio value update
    portfolio_value *= (1 + day_return / 100)

    # Save per-day
    date_to_returns[date] = day_return
    date_to_trades[date] = num_trades

    # Monthly aggregation
    month_key = pd.to_datetime(date).strftime("%Y-%m")
    monthly_returns[month_key] += day_return
    monthly_trades[month_key] += num_trades

    # One-line summary per day
    print(f"📅 {date}, 🧾 Trades: {num_trades}, 💰 Return: {day_return:.2f}%")

# === Monthly Summary ===
print("\n📊 Monthly Summary:")
for month in sorted(monthly_returns):
    print(f"🗓️ {month} — Trades: {monthly_trades[month]}, Return: {monthly_returns[month]:.2f}%")

# === Overall Summary ===
print("\n💼 Total Summary:")
print(f"🧮 Total Trades: {sum(monthly_trades.values())}")
print(f"💵 Total Return: {sum(monthly_returns.values()):.2f}%")
print(f"📈 Final Portfolio Value: ₹{round(portfolio_value, 2)}")

# Plotting & Analysis

In [ ]:
# 1% range & 1% entry & 0.75% body & NO SLIPPAGE

import pandas as pd
import matplotlib.pyplot as plt
strategy1_data = {
    "2024-11-01": 0.00,
    "2024-11-04": 1.53,"2024-11-05": 10.06,"2024-11-06": 2.14,"2024-11-07": 7.43,"2024-11-08": -2.47,
    "2024-11-11": -1.00,"2024-11-12": -1.76,"2024-11-13": -1.00,"2024-11-14": -1.58,"2024-11-18": -1.72,
    "2024-11-19": -5.13,"2024-11-21": -0.65,"2024-11-22": -3.53,"2024-11-25": -1.00,
    "2024-11-26": -0.98,"2024-11-27": 1.15,"2024-11-28": -0.31, "2024-11-29": 3.98,
    "2024-12-02": 4.50,"2024-12-03": 6.56,"2024-12-04": 1.39,"2024-12-05": 0.83,
    "2024-12-06": 5.70,"2024-12-09": 0.60,"2024-12-10": -0.65,
    "2024-12-11": 0.37,"2024-12-12": -1.73,"2024-12-13": -0.21,"2024-12-16": 2.82,"2024-12-17": 0.00,"2024-12-18": -2.48,
    "2024-12-19": -3.53,"2024-12-20": 0.00,"2024-12-23": 1.26,"2024-12-24": 4.37,"2024-12-26": 4.23,
    "2024-12-27": 4.34,"2024-12-30": 5.20,"2024-12-31": 10.48,

    '2025-01-01': 6.98, '2025-01-02': -5.08, '2025-01-03': 0.17, '2025-01-06': 0.00,
    '2025-01-07': -1.00, '2025-01-08': 7.02, '2025-01-09': -2.71, '2025-01-10': 0.00,
    '2025-01-13': 0.00, '2025-01-14': 8.59, '2025-01-15': -1.00, '2025-01-16': 0.02,
    '2025-01-17': -4.25, '2025-01-20': -4.45, '2025-01-21': 2.70, '2025-01-22': 0.63,
    '2025-01-23': -1.00, '2025-01-24': -3.00, '2025-01-27': 9.79, '2025-01-28': 1.81,
    '2025-01-29': 3.21, '2025-01-30': -4.04, '2025-01-31': 3.25, '2025-02-01': 5.46,
    '2025-02-03': -1.00, '2025-02-04': 5.77, '2025-02-05': -1.64, '2025-02-06': -0.67,
    '2025-02-07': -0.30, '2025-02-10': 2.79, '2025-02-11': 0.00, '2025-02-12': 0.00,
    '2025-02-13': -2.00, '2025-02-14': 0.00, '2025-02-17': 0.00, '2025-02-18': -0.07,
    '2025-02-19': 10.64, '2025-02-20': -1.77, '2025-02-21': 0.00, '2025-02-24': 3.87,
    '2025-02-25': 0.00, '2025-02-27': 10.62, '2025-02-28': -0.02,

    '2025-03-03': 0.00, '2025-03-04': -2.32, '2025-03-05': 0.40, '2025-03-06': -2.20,
    '2025-03-07': -1.36, '2025-03-10': -1.00, '2025-03-11': 6.51, '2025-03-12': -0.27,
    '2025-03-13': 4.03, '2025-03-17': 0.60, '2025-03-18': -0.47, '2025-03-19': 0.18,
    '2025-03-20': 0.53, '2025-03-21': 7.88, '2025-03-24': -1.65, '2025-03-25': 2.39,
    '2025-03-26': 0.00, '2025-03-27': 12.19, '2025-03-28': -3.00,

    '2025-04-01': -3.00, '2025-04-02': -2.24, '2025-04-03': 3.63, '2025-04-04': -1.00,
    '2025-04-07': 1.39, '2025-04-08': -3.33, '2025-04-09': 3.45, '2025-04-11': -1.90,
    '2025-04-15': 6.19, '2025-04-16': -1.37, '2025-04-17': -4.14, '2025-04-21': -0.41,
    '2025-04-22': -1.95, '2025-04-23': 5.33, '2025-04-24': 1.65, '2025-04-25': 2.09,
    '2025-04-28': -0.36, '2025-04-29': 0.38, '2025-04-30': -2.00,
    '2025-05-02': -0.18, '2025-05-05': -0.00, '2025-05-06': 0.00, '2025-05-07': 4.83,
    '2025-05-08': -3.00, '2025-05-09': 6.86, '2025-05-12': 2.09, '2025-05-13': 4.37,
    '2025-05-14': 0.91, '2025-05-15': 6.35, '2025-05-16': 4.18, '2025-05-19': -1.00,
    '2025-05-20': 5.95, '2025-05-21': 2.59, '2025-05-22': 9.65, '2025-05-23': -3.95,
    '2025-05-26': -1.07, '2025-05-27': -2.55, '2025-05-28': 2.58, '2025-05-29': 1.21,
    '2025-05-30': 20.66,

    '2025-06-02': 2.13, '2025-06-03': -2.31, '2025-06-04': 0.19, '2025-06-05': -0.11,
    '2025-06-06': -2.79, '2025-06-09': 1.74, '2025-06-10': 2.04, '2025-06-11': -2.63,
    '2025-06-12': -1.00, '2025-06-13': 3.19, '2025-06-16': 4.96, '2025-06-17': 0.00,
    '2025-06-18': -1.65, '2025-06-19': 0.00, '2025-06-20': -1.86, '2025-06-23': 5.91,
    '2025-06-24': -2.10, '2025-06-25': 4.43, '2025-06-26': 0.61, '2025-06-27': -3.00,
    '2025-06-30': 4.11,

    '2025-07-01': 9.35, '2025-07-02': 0.17, '2025-07-03': 2.73, '2025-07-04': -0.06,
    '2025-07-07': 0.93, '2025-07-08': -1.79, '2025-07-09': -1.96, '2025-07-10': -0.28,
    '2025-07-11': -1.05, '2025-07-14': 1.34
}

strategy1_trades = {
    "2024-11-01": 0,
    "2024-11-04": 2,"2024-11-05": 8,"2024-11-06": 10,"2024-11-07": 3,"2024-11-08": 3,"2024-11-11": 1,
    "2024-11-12": 3,"2024-11-13": 1,"2024-11-14": 2,"2024-11-18": 3,
    "2024-11-19": 6,"2024-11-21": 4,"2024-11-22": 5,"2024-11-25": 1,
    "2024-11-26": 10,"2024-11-27": 8,"2024-11-28": 1,"2024-11-29": 9,

    "2024-12-02": 8,"2024-12-03": 13,"2024-12-04": 1,"2024-12-05": 4,"2024-12-06": 4,
    "2024-12-09": 3,"2024-12-10": 5,"2024-12-11": 10,"2024-12-12": 3,
    "2024-12-13": 2,"2024-12-16": 6,"2024-12-17": 0,"2024-12-18": 6,
    "2024-12-19": 8,"2024-12-20": 0,"2024-12-23": 4,"2024-12-24": 5,
    "2024-12-26": 8,"2024-12-27": 8,"2024-12-30": 3,"2024-12-31": 12,
    '2025-01-01': 10, '2025-01-02': 13, '2025-01-03': 5, '2025-01-06': 0,
    '2025-01-07': 1, '2025-01-08': 4, '2025-01-09': 4, '2025-01-10': 0,
    '2025-01-13': 0, '2025-01-14': 2, '2025-01-15': 1, '2025-01-16': 10,
    '2025-01-17': 7, '2025-01-20': 10, '2025-01-21': 3, '2025-01-22': 1,
    '2025-01-23': 4, '2025-01-24': 3, '2025-01-27': 3, '2025-01-28': 5,
    '2025-01-29': 4, '2025-01-30': 5, '2025-01-31': 6, '2025-02-01': 2,
    '2025-02-03': 1, '2025-02-04': 9, '2025-02-05': 3, '2025-02-06': 6,
    '2025-02-07': 3, '2025-02-10': 2, '2025-02-11': 0, '2025-02-12': 0,
    '2025-02-13': 2, '2025-02-14': 0, '2025-02-17': 0, '2025-02-18': 3,
    '2025-02-19': 2, '2025-02-20': 5, '2025-02-21': 0, '2025-02-24': 3,
    '2025-02-25': 0, '2025-02-27': 6, '2025-02-28': 3,'2025-03-03': 0,
    '2025-03-04': 5, '2025-03-05': 4, '2025-03-06': 4,
    '2025-03-07': 6, '2025-03-10': 1, '2025-03-11': 5, '2025-03-12': 2,
    '2025-03-13': 4, '2025-03-17': 1, '2025-03-18': 12, '2025-03-19': 14,
    '2025-03-20': 3, '2025-03-21': 7, '2025-03-24': 5, '2025-03-25': 1,
    '2025-03-26': 0, '2025-03-27': 16, '2025-03-28': 3,

    '2025-04-01': 3, '2025-04-02': 5, '2025-04-03': 14, '2025-04-04': 1,
    '2025-04-07': 1, '2025-04-08': 5, '2025-04-09': 4, '2025-04-11': 10,
    '2025-04-15': 7, '2025-04-16': 2, '2025-04-17': 15, '2025-04-21': 13,
    '2025-04-22': 8, '2025-04-23': 7, '2025-04-24': 3, '2025-04-25': 1,
    '2025-04-28': 6, '2025-04-29': 3, '2025-04-30': 2,'2025-05-02': 3,
    '2025-05-05': 11, '2025-05-06': 0, '2025-05-07': 4,
    '2025-05-08': 3, '2025-05-09': 7, '2025-05-12': 10, '2025-05-13': 2,
    '2025-05-14': 4, '2025-05-15': 13, '2025-05-16': 11, '2025-05-19': 1,
    '2025-05-20': 3, '2025-05-21': 2, '2025-05-22': 6, '2025-05-23': 8,
    '2025-05-26': 9, '2025-05-27': 4, '2025-05-28': 6, '2025-05-29': 6,
    '2025-05-30': 15,

    '2025-06-02': 8, '2025-06-03': 3, '2025-06-04': 13, '2025-06-05': 11,
    '2025-06-06': 9, '2025-06-09': 12, '2025-06-10': 11, '2025-06-11': 6,
    '2025-06-12': 1, '2025-06-13': 6, '2025-06-16': 9, '2025-06-17': 0,
    '2025-06-18': 3, '2025-06-19': 0, '2025-06-20': 4, '2025-06-23': 12,
    '2025-06-24': 4, '2025-06-25': 6, '2025-06-26': 5, '2025-06-27': 3,
    '2025-06-30': 5,

    '2025-07-01': 6, '2025-07-02': 10, '2025-07-03': 4, '2025-07-04': 2,
    '2025-07-07': 4, '2025-07-08': 5, '2025-07-09': 7, '2025-07-10': 2,
    '2025-07-11': 7, '2025-07-14': 4
}


# ==== Strategy Processing Function ====
def process_strategy(data_dict, trades_dict, cost_per_trade=40, initial_capital=100000, leverage=5, trades_per_day=10):
    df = pd.DataFrame({
        'Return (%)': data_dict,
        'Trades': trades_dict
    })
    df.index = pd.to_datetime(df.index)
    df.sort_index(inplace=True)

    df['Adjusted Return (%)'] = df['Return (%)'] * (1 / trades_per_day) * leverage
    df['Cost (₹)'] = df['Trades'] * cost_per_trade
    df['Net Return (%)'] = df['Adjusted Return (%)'] - (df['Cost (₹)'] / initial_capital * 100)

    df['Capital'] = initial_capital
    for i in range(1, len(df)):
        prev_cap = df.iloc[i - 1]['Capital']
        net_ret = df.iloc[i]['Net Return (%)']
        df.iloc[i, df.columns.get_loc('Capital')] = prev_cap * (1 + net_ret / 100)

    df['Peak'] = df['Capital'].cummax()
    df['Drawdown (₹)'] = df['Capital'] - df['Peak']
    df['Drawdown (%)'] = df['Drawdown (₹)'] / df['Peak'] * 100

    return df, df['Drawdown (%)'].min()

# ==== Process ====
df1, max_dd1 = process_strategy(strategy1_data, strategy1_trades)

# ==== Max Drawdown Print ====
print(f"\n📉 Strategy Max Drawdown: {max_dd1:.2f}%")

# ==== Capital Curve Plot ====
plt.figure(figsize=(20, 8))
plt.plot(df1.index, df1['Capital'], label='Capital', linewidth=2, color='blue')
plt.axhline(100000, color='gray', linestyle='--', label='Initial Capital')
for i, date in enumerate(df1.index):
    plt.text(date, df1['Capital'].iloc[i] + 200, f"{df1['Capital'].iloc[i]:,.0f}", fontsize=8, ha='center', color='blue')
plt.title('Capital Curve (₹10K per Trade, 4x Leverage, ₹40 Cost/Trade)')
plt.ylabel('Capital (₹)')
plt.xlabel('Date')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# ==== Drawdown Plot ====
plt.figure(figsize=(14, 5))
plt.plot(df1.index, df1['Drawdown (%)'], label='Drawdown', linestyle='--', color='red')
plt.axhline(0, color='black', linestyle=':')
plt.title('Drawdown Over Time (%)')
plt.ylabel('Drawdown (%)')
plt.xlabel('Date')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


# ==== Monthly Summary ====
monthly_summary = df1.groupby(df1.index.to_period("M")).agg({
    'Net Return (%)': 'sum',
    'Trades': 'sum',
    'Capital': 'last'
}).rename(columns={
    'Net Return (%)': 'Monthly Return (%)',
    'Trades': 'Monthly Trades',
    'Capital': 'End Capital (₹)'
})

# Convert PeriodIndex to string for clean print
monthly_summary.index = monthly_summary.index.astype(str)

print("\n📊 Monthly Performance Summary:")
for month, row in monthly_summary.iterrows():
    print(f"🗓️ {month} — Trades: {int(row['Monthly Trades'])}, "
          f"Return: {row['Monthly Return (%)']:.2f}%, "
          f"Capital: ₹{row['End Capital (₹)']:.2f}")

In [ ]:
# 1% range & 1% entry & 0.75% body & SLIPPAGE

# ******************************************* VERY IMP IMP

import pandas as pd
import matplotlib.pyplot as plt
strategy1_data = {
    "2024-08-01": -1.00, "2024-08-02": 1.77, "2024-08-05": 0.00, "2024-08-06": -1.00, "2024-08-07": 1.23,
    "2024-08-08": 1.30, "2024-08-09": -3.30, "2024-08-12": 4.48, "2024-08-13": -1.00, "2024-08-14": 0.02,
    "2024-08-16": 10.07, "2024-08-19": 1.48, "2024-08-20": 7.84, "2024-08-21": 8.85, "2024-08-22": -0.40,
    "2024-08-23": 10.05, "2024-08-26": 5.66, "2024-08-27": 8.38, "2024-08-28": -1.85, "2024-08-29": -2.00,"2024-08-30": 2.86,

    "2024-09-02": -0.11, "2024-09-03": -7.97, "2024-09-04": 4.57, "2024-09-05": 0.86, "2024-09-06": -1.00,
    "2024-09-09": 0.50, "2024-09-10": 6.05, "2024-09-11": 9.39, "2024-09-12": 14.14, "2024-09-13": -8.29,
    "2024-09-16": -0.82, "2024-09-17": 9.08, "2024-09-18": 1.40, "2024-09-19": 0.24, "2024-09-20": 2.18,
    "2024-09-23": -4.65, "2024-09-24": 4.32, "2024-09-25": -0.39, "2024-09-26": -2.50, "2024-09-27": 0.14,"2024-09-30": 2.57,

    "2024-10-01": 8.06, "2024-10-03": -4.31, "2024-10-04": 0.36, "2024-10-07": 0.00, "2024-10-08": -0.88,
    "2024-10-09": 3.17, "2024-10-10": 0.61, "2024-10-11": -1.86, "2024-10-14": -0.40, "2024-10-15": 0.96,
    "2024-10-16": 2.36, "2024-10-17": 4.49, "2024-10-18": -6.66, "2024-10-21": -2.39, "2024-10-22": 1.93,
    "2024-10-23": 0.77, "2024-10-24": -2.00, "2024-10-25": 1.84, "2024-10-28": -0.21, "2024-10-29": 12.44,
    "2024-10-30": -0.89, "2024-10-31": 0.62,

    "2024-11-01": 0.00,"2024-11-04": 1.53,"2024-11-05": 10.06,"2024-11-06": 2.14,"2024-11-07": 7.43,"2024-11-08": -2.47,
    "2024-11-11": -1.00,"2024-11-12": -1.76,"2024-11-13": -1.00,"2024-11-14": -1.58,"2024-11-18": -1.72,
    "2024-11-19": -5.13,"2024-11-21": -0.65,"2024-11-22": -3.53,"2024-11-25": -1.00,
    "2024-11-26": -0.98,"2024-11-27": 1.15,"2024-11-28": -0.31, "2024-11-29": 3.98,

    "2024-12-02": 4.50,"2024-12-03": 6.56,"2024-12-04": 1.39,"2024-12-05": 0.83,
    "2024-12-06": 5.70,"2024-12-09": 0.60,"2024-12-10": -0.65,
    "2024-12-11": 0.37,"2024-12-12": -1.73,"2024-12-13": -0.21,"2024-12-16": 2.82,"2024-12-17": 0.00,"2024-12-18": -2.48,
    "2024-12-19": -3.53,"2024-12-20": 0.00,"2024-12-23": 1.26,"2024-12-24": 4.37,"2024-12-26": 4.23,
    "2024-12-27": 4.34,"2024-12-30": 5.20,"2024-12-31": 10.48,

    '2025-01-01': 6.98, '2025-01-02': -5.08, '2025-01-03': 0.17, '2025-01-06': 0.00,
    '2025-01-07': -1.00, '2025-01-08': 7.02, '2025-01-09': -2.71, '2025-01-10': 0.00,
    '2025-01-13': 0.00, '2025-01-14': 8.59, '2025-01-15': -1.00, '2025-01-16': 0.02,
    '2025-01-17': -4.25, '2025-01-20': -4.45, '2025-01-21': 2.70, '2025-01-22': 0.63,
    '2025-01-23': -1.00, '2025-01-24': -3.00, '2025-01-27': 9.79, '2025-01-28': 1.81,
    '2025-01-29': 3.21, '2025-01-30': -4.04, '2025-01-31': 3.25,

    '2025-02-01': 5.46,'2025-02-03': -1.00, '2025-02-04': 5.77, '2025-02-05': -1.64, '2025-02-06': -0.67,
    '2025-02-07': -0.30, '2025-02-10': 2.79, '2025-02-11': 0.00, '2025-02-12': 0.00,
    '2025-02-13': -2.00, '2025-02-14': 0.00, '2025-02-17': 0.00, '2025-02-18': -0.07,
    '2025-02-19': 10.64, '2025-02-20': -1.77, '2025-02-21': 0.00, '2025-02-24': 3.87,
    '2025-02-25': 0.00, '2025-02-27': 10.62, '2025-02-28': -0.02,

    '2025-03-03': 0.00, '2025-03-04': -2.32, '2025-03-05': 0.40, '2025-03-06': -2.20,
    '2025-03-07': -1.36, '2025-03-10': -1.00, '2025-03-11': 6.51, '2025-03-12': -0.27,
    '2025-03-13': 4.03, '2025-03-17': 0.60, '2025-03-18': -0.47, '2025-03-19': 0.18,
    '2025-03-20': 0.53, '2025-03-21': 7.88, '2025-03-24': -1.65, '2025-03-25': 2.39,
    '2025-03-26': 0.00, '2025-03-27': 12.19, '2025-03-28': -3.00,

    '2025-04-01': -3.00, '2025-04-02': -2.24, '2025-04-03': 3.63, '2025-04-04': -1.00,
    '2025-04-07': 1.39, '2025-04-08': -3.33, '2025-04-09': 3.45, '2025-04-11': -1.90,
    '2025-04-15': 6.19, '2025-04-16': -1.37, '2025-04-17': -4.14, '2025-04-21': -0.41,
    '2025-04-22': -1.95, '2025-04-23': 5.33, '2025-04-24': 1.65, '2025-04-25': 2.09,
    '2025-04-28': -0.36, '2025-04-29': 0.38, '2025-04-30': -2.00,

    '2025-05-02': -0.18, '2025-05-05': -0.00, '2025-05-06': 0.00, '2025-05-07': 4.83,
    '2025-05-08': -3.00, '2025-05-09': 6.86, '2025-05-12': 2.09, '2025-05-13': 4.37,
    '2025-05-14': 0.91, '2025-05-15': 6.35, '2025-05-16': 4.18, '2025-05-19': -1.00,
    '2025-05-20': 5.95, '2025-05-21': 2.59, '2025-05-22': 9.65, '2025-05-23': -3.95,
    '2025-05-26': -1.07, '2025-05-27': -2.55, '2025-05-28': 2.58, '2025-05-29': 1.21,
    '2025-05-30': 20.66,

    '2025-06-02': 2.13, '2025-06-03': -2.31, '2025-06-04': 0.19, '2025-06-05': -0.11,
    '2025-06-06': -2.79, '2025-06-09': 1.74, '2025-06-10': 2.04, '2025-06-11': -2.63,
    '2025-06-12': -1.00, '2025-06-13': 3.19, '2025-06-16': 4.96, '2025-06-17': 0.00,
    '2025-06-18': -1.65, '2025-06-19': 0.00, '2025-06-20': -1.86, '2025-06-23': 5.91,
    '2025-06-24': -2.10, '2025-06-25': 4.43, '2025-06-26': 0.61, '2025-06-27': -3.00,
    '2025-06-30': 4.11,

    '2025-07-01': 9.35, '2025-07-02': 0.17, '2025-07-03': 2.73, '2025-07-04': -0.06,
    '2025-07-07': 0.93, '2025-07-08': -1.79, '2025-07-09': -1.96, '2025-07-10': -0.28,
    '2025-07-11': -1.05, '2025-07-14': 1.34
}

strategy1_trades = {
    "2024-08-01": 1, "2024-08-02": 6, "2024-08-05": 0, "2024-08-06": 1, "2024-08-07": 8,
    "2024-08-08": 4, "2024-08-09": 4, "2024-08-12": 5, "2024-08-13": 1, "2024-08-14": 5,
    "2024-08-16": 12, "2024-08-19": 3, "2024-08-20": 9, "2024-08-21": 8, "2024-08-22": 3,
    "2024-08-23": 4, "2024-08-26": 5, "2024-08-27": 5, "2024-08-28": 3, "2024-08-29": 2,"2024-08-30": 10,

    "2024-09-02": 2, "2024-09-03": 10, "2024-09-04": 8, "2024-09-05": 7, "2024-09-06": 1,
    "2024-09-09": 6, "2024-09-10": 7, "2024-09-11": 7, "2024-09-12": 11, "2024-09-13": 11,
    "2024-09-16": 10, "2024-09-17": 12, "2024-09-18": 6, "2024-09-19": 1, "2024-09-20": 11,
    "2024-09-23": 7, "2024-09-24": 5, "2024-09-25": 4, "2024-09-26": 4, "2024-09-27": 10,"2024-09-30": 4,

    "2024-10-01": 8, "2024-10-03": 5, "2024-10-04": 2, "2024-10-07": 0, "2024-10-08": 2,
    "2024-10-09": 3, "2024-10-10": 6, "2024-10-11": 5, "2024-10-14": 11, "2024-10-15": 6,
    "2024-10-16": 8, "2024-10-17": 3, "2024-10-18": 8, "2024-10-21": 5, "2024-10-22": 1,
    "2024-10-23": 3, "2024-10-24": 2, "2024-10-25": 2, "2024-10-28": 5, "2024-10-29": 10,
    "2024-10-30": 4, "2024-10-31": 3,

    "2024-11-01": 0,"2024-11-04": 2,"2024-11-05": 8,"2024-11-06": 10,"2024-11-07": 3,"2024-11-08": 3,"2024-11-11": 1,
    "2024-11-12": 3,"2024-11-13": 1,"2024-11-14": 2,"2024-11-18": 3,
    "2024-11-19": 6,"2024-11-21": 4,"2024-11-22": 5,"2024-11-25": 1,
    "2024-11-26": 10,"2024-11-27": 8,"2024-11-28": 1,"2024-11-29": 9,

    "2024-12-02": 8,"2024-12-03": 13,"2024-12-04": 1,"2024-12-05": 4,"2024-12-06": 4,
    "2024-12-09": 3,"2024-12-10": 5,"2024-12-11": 10,"2024-12-12": 3,
    "2024-12-13": 2,"2024-12-16": 6,"2024-12-17": 0,"2024-12-18": 6,
    "2024-12-19": 8,"2024-12-20": 0,"2024-12-23": 4,"2024-12-24": 5,
    "2024-12-26": 8,"2024-12-27": 8,"2024-12-30": 3,"2024-12-31": 12,

    '2025-01-01': 10, '2025-01-02': 13, '2025-01-03': 5, '2025-01-06': 0,
    '2025-01-07': 1, '2025-01-08': 4, '2025-01-09': 4, '2025-01-10': 0,
    '2025-01-13': 0, '2025-01-14': 2, '2025-01-15': 1, '2025-01-16': 10,
    '2025-01-17': 7, '2025-01-20': 10, '2025-01-21': 3, '2025-01-22': 1,
    '2025-01-23': 4, '2025-01-24': 3, '2025-01-27': 3, '2025-01-28': 5,
    '2025-01-29': 4, '2025-01-30': 5, '2025-01-31': 6,

    '2025-02-01': 2,'2025-02-03': 1, '2025-02-04': 9, '2025-02-05': 3, '2025-02-06': 6,
    '2025-02-07': 3, '2025-02-10': 2, '2025-02-11': 0, '2025-02-12': 0,
    '2025-02-13': 2, '2025-02-14': 0, '2025-02-17': 0, '2025-02-18': 3,
    '2025-02-19': 2, '2025-02-20': 5, '2025-02-21': 0, '2025-02-24': 3,
    '2025-02-25': 0, '2025-02-27': 6, '2025-02-28': 3,

    '2025-03-03': 0, '2025-03-04': 5, '2025-03-05': 4, '2025-03-06': 4,
    '2025-03-07': 6, '2025-03-10': 1, '2025-03-11': 5, '2025-03-12': 2,
    '2025-03-13': 4, '2025-03-17': 1, '2025-03-18': 12, '2025-03-19': 14,
    '2025-03-20': 3, '2025-03-21': 7, '2025-03-24': 5, '2025-03-25': 1,
    '2025-03-26': 0, '2025-03-27': 16, '2025-03-28': 3,

    '2025-04-01': 3, '2025-04-02': 5, '2025-04-03': 14, '2025-04-04': 1,
    '2025-04-07': 1, '2025-04-08': 5, '2025-04-09': 4, '2025-04-11': 10,
    '2025-04-15': 7, '2025-04-16': 2, '2025-04-17': 15, '2025-04-21': 13,
    '2025-04-22': 8, '2025-04-23': 7, '2025-04-24': 3, '2025-04-25': 1,
    '2025-04-28': 6, '2025-04-29': 3, '2025-04-30': 2,

    '2025-05-02': 3, '2025-05-05': 11, '2025-05-06': 0, '2025-05-07': 4,
    '2025-05-08': 3, '2025-05-09': 7, '2025-05-12': 10, '2025-05-13': 2,
    '2025-05-14': 4, '2025-05-15': 13, '2025-05-16': 11, '2025-05-19': 1,
    '2025-05-20': 3, '2025-05-21': 2, '2025-05-22': 6, '2025-05-23': 8,
    '2025-05-26': 9, '2025-05-27': 4, '2025-05-28': 6, '2025-05-29': 6,
    '2025-05-30': 15,

    '2025-06-02': 8, '2025-06-03': 3, '2025-06-04': 13, '2025-06-05': 11,
    '2025-06-06': 9, '2025-06-09': 12, '2025-06-10': 11, '2025-06-11': 6,
    '2025-06-12': 1, '2025-06-13': 6, '2025-06-16': 9, '2025-06-17': 0,
    '2025-06-18': 3, '2025-06-19': 0, '2025-06-20': 4, '2025-06-23': 12,
    '2025-06-24': 4, '2025-06-25': 6, '2025-06-26': 5, '2025-06-27': 3,
    '2025-06-30': 5,

    '2025-07-01': 6, '2025-07-02': 10, '2025-07-03': 4, '2025-07-04': 2,
    '2025-07-07': 4, '2025-07-08': 5, '2025-07-09': 7, '2025-07-10': 2,
    '2025-07-11': 7, '2025-07-14': 4
}

def process_strategy(data_dict, trades_dict, cost_per_trade=40, initial_capital=100000, leverage=5, trades_per_day=10, slippage_pct=0.2):
    df = pd.DataFrame({
        'Return (%)': data_dict,
        'Trades': trades_dict
    })
    df.index = pd.to_datetime(df.index)
    df.sort_index(inplace=True)

    # Adjusted returns with leverage
    df['Adjusted Return (%)'] = df['Return (%)'] * (1 / trades_per_day) * leverage

    # Include cost + slippage
    slippage_total_pct = 1 * slippage_pct / 100  # 0.5% per side = 1% round trip
    df['Cost (₹)'] = df['Trades'] * cost_per_trade + (initial_capital * slippage_total_pct * (df['Trades'] / trades_per_day))

    # Net return after cost
    df['Net Return (%)'] = df['Adjusted Return (%)'] - (df['Cost (₹)'] / initial_capital * 100)

    # Capital calculation
    df['Capital'] = initial_capital
    for i in range(1, len(df)):
        prev_cap = df.iloc[i - 1]['Capital']
        net_ret = df.iloc[i]['Net Return (%)']
        df.iloc[i, df.columns.get_loc('Capital')] = prev_cap * (1 + net_ret / 100)

    # Drawdown
    df['Peak'] = df['Capital'].cummax()
    df['Drawdown (₹)'] = df['Capital'] - df['Peak']
    df['Drawdown (%)'] = df['Drawdown (₹)'] / df['Peak'] * 100

    return df, df['Drawdown (%)'].min()


# ==== Process ====
df1, max_dd1 = process_strategy(strategy1_data, strategy1_trades)

# ==== Max Drawdown Print ====
print(f"\n📉 Strategy Max Drawdown: {max_dd1:.2f}%")

# ==== Capital Curve Plot ====
plt.figure(figsize=(20, 8))
plt.plot(df1.index, df1['Capital'], label='Capital', linewidth=2, color='blue')
plt.axhline(100000, color='gray', linestyle='--', label='Initial Capital')
for i, date in enumerate(df1.index):
    plt.text(date, df1['Capital'].iloc[i] + 200, f"{df1['Capital'].iloc[i]:,.0f}", fontsize=8, ha='center', color='blue')
plt.title('Capital Curve (₹10K per Trade, 4x Leverage, ₹40 Cost/Trade)')
plt.ylabel('Capital (₹)')
plt.xlabel('Date')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# ==== Drawdown Plot ====
plt.figure(figsize=(14, 5))
plt.plot(df1.index, df1['Drawdown (%)'], label='Drawdown', linestyle='--', color='red')
plt.axhline(0, color='black', linestyle=':')
plt.title('Drawdown Over Time (%)')
plt.ylabel('Drawdown (%)')
plt.xlabel('Date')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


# ==== Monthly Summary ====
monthly_summary = df1.groupby(df1.index.to_period("M")).agg({
    'Net Return (%)': 'sum',
    'Trades': 'sum',
    'Capital': 'last'
}).rename(columns={
    'Net Return (%)': 'Monthly Return (%)',
    'Trades': 'Monthly Trades',
    'Capital': 'End Capital (₹)'
})

# Convert PeriodIndex to string for clean print
monthly_summary.index = monthly_summary.index.astype(str)

print("\n📊 Monthly Performance Summary:")
for month, row in monthly_summary.iterrows():
    print(f"🗓️ {month} — Trades: {int(row['Monthly Trades'])}, "
          f"Return: {row['Monthly Return (%)']:.2f}%, "
          f"Capital: ₹{row['End Capital (₹)']:.2f}")

In [ ]:
#RISK METRICS____________________________________________________________

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
import random

strategy1_data = {"2024-11-01": 0.00,
    "2024-11-04": 1.53,"2024-11-05": 10.06,"2024-11-06": 2.14,"2024-11-07": 7.43,"2024-11-08": -2.47,
    "2024-11-11": -1.00,"2024-11-12": -1.76,"2024-11-13": -1.00,"2024-11-14": -1.58,"2024-11-18": -1.72,
    "2024-11-19": -5.13,"2024-11-21": -0.65,"2024-11-22": -3.53,"2024-11-25": -1.00,
    "2024-11-26": -0.98,"2024-11-27": 1.15,"2024-11-28": -0.31, "2024-11-29": 3.98,
    "2024-12-02": 4.50,"2024-12-03": 6.56,"2024-12-04": 1.39,"2024-12-05": 0.83,
    "2024-12-06": 5.70,"2024-12-09": 0.60,"2024-12-10": -0.65,
    "2024-12-11": 0.37,"2024-12-12": -1.73,"2024-12-13": -0.21,"2024-12-16": 2.82,"2024-12-17": 0.00,"2024-12-18": -2.48,
    "2024-12-19": -3.53,"2024-12-20": 0.00,"2024-12-23": 1.26,"2024-12-24": 4.37,"2024-12-26": 4.23,
    "2024-12-27": 4.34,"2024-12-30": 5.20,"2024-12-31": 10.48,

    '2025-01-01': 6.98, '2025-01-02': -5.08, '2025-01-03': 0.17, '2025-01-06': 0.00,
    '2025-01-07': -1.00, '2025-01-08': 7.02, '2025-01-09': -2.71, '2025-01-10': 0.00,
    '2025-01-13': 0.00, '2025-01-14': 8.59, '2025-01-15': -1.00, '2025-01-16': 0.02,
    '2025-01-17': -4.25, '2025-01-20': -4.45, '2025-01-21': 2.70, '2025-01-22': 0.63,
    '2025-01-23': -1.00, '2025-01-24': -3.00, '2025-01-27': 9.79, '2025-01-28': 1.81,
    '2025-01-29': 3.21, '2025-01-30': -4.04, '2025-01-31': 3.25, '2025-02-01': 5.46,
    '2025-02-03': -1.00, '2025-02-04': 5.77, '2025-02-05': -1.64, '2025-02-06': -0.67,
    '2025-02-07': -0.30, '2025-02-10': 2.79, '2025-02-11': 0.00, '2025-02-12': 0.00,
    '2025-02-13': -2.00, '2025-02-14': 0.00, '2025-02-17': 0.00, '2025-02-18': -0.07,
    '2025-02-19': 10.64, '2025-02-20': -1.77, '2025-02-21': 0.00, '2025-02-24': 3.87,
    '2025-02-25': 0.00, '2025-02-27': 10.62, '2025-02-28': -0.02,

    '2025-03-03': 0.00, '2025-03-04': -2.32, '2025-03-05': 0.40, '2025-03-06': -2.20,
    '2025-03-07': -1.36, '2025-03-10': -1.00, '2025-03-11': 6.51, '2025-03-12': -0.27,
    '2025-03-13': 4.03, '2025-03-17': 0.60, '2025-03-18': -0.47, '2025-03-19': 0.18,
    '2025-03-20': 0.53, '2025-03-21': 7.88, '2025-03-24': -1.65, '2025-03-25': 2.39,
    '2025-03-26': 0.00, '2025-03-27': 12.19, '2025-03-28': -3.00,

    '2025-04-01': -3.00, '2025-04-02': -2.24, '2025-04-03': 3.63, '2025-04-04': -1.00,
    '2025-04-07': 1.39, '2025-04-08': -3.33, '2025-04-09': 3.45, '2025-04-11': -1.90,
    '2025-04-15': 6.19, '2025-04-16': -1.37, '2025-04-17': -4.14, '2025-04-21': -0.41,
    '2025-04-22': -1.95, '2025-04-23': 5.33, '2025-04-24': 1.65, '2025-04-25': 2.09,
    '2025-04-28': -0.36, '2025-04-29': 0.38, '2025-04-30': -2.00,
    '2025-05-02': -0.18, '2025-05-05': -0.00, '2025-05-06': 0.00, '2025-05-07': 4.83,
    '2025-05-08': -3.00, '2025-05-09': 6.86, '2025-05-12': 2.09, '2025-05-13': 4.37,
    '2025-05-14': 0.91, '2025-05-15': 6.35, '2025-05-16': 4.18, '2025-05-19': -1.00,
    '2025-05-20': 5.95, '2025-05-21': 2.59, '2025-05-22': 9.65, '2025-05-23': -3.95,
    '2025-05-26': -1.07, '2025-05-27': -2.55, '2025-05-28': 2.58, '2025-05-29': 1.21,
    '2025-05-30': 20.66,

    '2025-06-02': 2.13, '2025-06-03': -2.31, '2025-06-04': 0.19, '2025-06-05': -0.11,
    '2025-06-06': -2.79, '2025-06-09': 1.74, '2025-06-10': 2.04, '2025-06-11': -2.63,
    '2025-06-12': -1.00, '2025-06-13': 3.19, '2025-06-16': 4.96, '2025-06-17': 0.00,
    '2025-06-18': -1.65, '2025-06-19': 0.00, '2025-06-20': -1.86, '2025-06-23': 5.91,
    '2025-06-24': -2.10, '2025-06-25': 4.43, '2025-06-26': 0.61, '2025-06-27': -3.00,
    '2025-06-30': 4.11,

    '2025-07-01': 9.35, '2025-07-02': 0.17, '2025-07-03': 2.73, '2025-07-04': -0.06,
    '2025-07-07': 0.93, '2025-07-08': -1.79, '2025-07-09': -1.96, '2025-07-10': -0.28,
    '2025-07-11': -1.05, '2025-07-14': 1.34
}

strategy1_trades = {
    "2024-11-01": 0,
    "2024-11-04": 2,"2024-11-05": 8,"2024-11-06": 10,"2024-11-07": 3,"2024-11-08": 3,"2024-11-11": 1,
    "2024-11-12": 3,"2024-11-13": 1,"2024-11-14": 2,"2024-11-18": 3,
    "2024-11-19": 6,"2024-11-21": 4,"2024-11-22": 5,"2024-11-25": 1,
    "2024-11-26": 10,"2024-11-27": 8,"2024-11-28": 1,"2024-11-29": 9,

    "2024-12-02": 8,"2024-12-03": 13,"2024-12-04": 1,"2024-12-05": 4,"2024-12-06": 4,
    "2024-12-09": 3,"2024-12-10": 5,"2024-12-11": 10,"2024-12-12": 3,
    "2024-12-13": 2,"2024-12-16": 6,"2024-12-17": 0,"2024-12-18": 6,
    "2024-12-19": 8,"2024-12-20": 0,"2024-12-23": 4,"2024-12-24": 5,
    "2024-12-26": 8,"2024-12-27": 8,"2024-12-30": 3,"2024-12-31": 12,
    '2025-01-01': 10, '2025-01-02': 13, '2025-01-03': 5, '2025-01-06': 0,
    '2025-01-07': 1, '2025-01-08': 4, '2025-01-09': 4, '2025-01-10': 0,
    '2025-01-13': 0, '2025-01-14': 2, '2025-01-15': 1, '2025-01-16': 10,
    '2025-01-17': 7, '2025-01-20': 10, '2025-01-21': 3, '2025-01-22': 1,
    '2025-01-23': 4, '2025-01-24': 3, '2025-01-27': 3, '2025-01-28': 5,
    '2025-01-29': 4, '2025-01-30': 5, '2025-01-31': 6, '2025-02-01': 2,
    '2025-02-03': 1, '2025-02-04': 9, '2025-02-05': 3, '2025-02-06': 6,
    '2025-02-07': 3, '2025-02-10': 2, '2025-02-11': 0, '2025-02-12': 0,
    '2025-02-13': 2, '2025-02-14': 0, '2025-02-17': 0, '2025-02-18': 3,
    '2025-02-19': 2, '2025-02-20': 5, '2025-02-21': 0, '2025-02-24': 3,
    '2025-02-25': 0, '2025-02-27': 6, '2025-02-28': 3,'2025-03-03': 0,
    '2025-03-04': 5, '2025-03-05': 4, '2025-03-06': 4,
    '2025-03-07': 6, '2025-03-10': 1, '2025-03-11': 5, '2025-03-12': 2,
    '2025-03-13': 4, '2025-03-17': 1, '2025-03-18': 12, '2025-03-19': 14,
    '2025-03-20': 3, '2025-03-21': 7, '2025-03-24': 5, '2025-03-25': 1,
    '2025-03-26': 0, '2025-03-27': 16, '2025-03-28': 3,

    '2025-04-01': 3, '2025-04-02': 5, '2025-04-03': 14, '2025-04-04': 1,
    '2025-04-07': 1, '2025-04-08': 5, '2025-04-09': 4, '2025-04-11': 10,
    '2025-04-15': 7, '2025-04-16': 2, '2025-04-17': 15, '2025-04-21': 13,
    '2025-04-22': 8, '2025-04-23': 7, '2025-04-24': 3, '2025-04-25': 1,
    '2025-04-28': 6, '2025-04-29': 3, '2025-04-30': 2,'2025-05-02': 3,
    '2025-05-05': 11, '2025-05-06': 0, '2025-05-07': 4,
    '2025-05-08': 3, '2025-05-09': 7, '2025-05-12': 10, '2025-05-13': 2,
    '2025-05-14': 4, '2025-05-15': 13, '2025-05-16': 11, '2025-05-19': 1,
    '2025-05-20': 3, '2025-05-21': 2, '2025-05-22': 6, '2025-05-23': 8,
    '2025-05-26': 9, '2025-05-27': 4, '2025-05-28': 6, '2025-05-29': 6,
    '2025-05-30': 15,

    '2025-06-02': 8, '2025-06-03': 3, '2025-06-04': 13, '2025-06-05': 11,
    '2025-06-06': 9, '2025-06-09': 12, '2025-06-10': 11, '2025-06-11': 6,
    '2025-06-12': 1, '2025-06-13': 6, '2025-06-16': 9, '2025-06-17': 0,
    '2025-06-18': 3, '2025-06-19': 0, '2025-06-20': 4, '2025-06-23': 12,
    '2025-06-24': 4, '2025-06-25': 6, '2025-06-26': 5, '2025-06-27': 3,
    '2025-06-30': 5,

    '2025-07-01': 6, '2025-07-02': 10, '2025-07-03': 4, '2025-07-04': 2,
    '2025-07-07': 4, '2025-07-08': 5, '2025-07-09': 7, '2025-07-10': 2,
    '2025-07-11': 7, '2025-07-14': 4}


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

def process_strategy(data_dict, trades_dict, cost_per_trade=40, initial_capital=100000, leverage=5,
                     trades_per_day=10, slippage_pct=0.25, simulate_missed=False, skip_pct=0.1):

    df = pd.DataFrame({
        'Return (%)': data_dict,
        'Trades': trades_dict
    })
    df.index = pd.to_datetime(df.index)
    df.sort_index(inplace=True)

    # === Simulate Missed Trades ===
    if simulate_missed:
        skip_days = random.sample(list(df.index), int(len(df) * skip_pct))
        df.loc[skip_days, 'Return (%)'] = 0
        df.loc[skip_days, 'Trades'] = 0

    # === Net Return Computation ===
    df['Adjusted Return (%)'] = df['Return (%)'] * (1 / trades_per_day) * leverage
    slippage_total_pct = slippage_pct / 100
    df['Cost (₹)'] = df['Trades'] * cost_per_trade + (initial_capital * slippage_total_pct * (df['Trades'] / trades_per_day))
    df['Net Return (%)'] = df['Adjusted Return (%)'] - (df['Cost (₹)'] / initial_capital * 100)

    # === Capital Growth ===
    df['Capital'] = initial_capital
    for i in range(1, len(df)):
        prev_cap = df.iloc[i - 1]['Capital']
        net_ret = df.iloc[i]['Net Return (%)']
        df.iloc[i, df.columns.get_loc('Capital')] = prev_cap * (1 + net_ret / 100)

    # === Drawdown Metrics ===
    df['Peak'] = df['Capital'].cummax()
    df['Drawdown (₹)'] = df['Capital'] - df['Peak']
    df['Drawdown (%)'] = df['Drawdown (₹)'] / df['Peak'] * 100
    max_dd = df['Drawdown (%)'].min()

    # === Max Drawdown Duration ===
    drawdown_duration = (df['Drawdown (₹)'] < 0).astype(int)
    max_dd_duration = 0
    current = 0
    for d in drawdown_duration:
        if d:
            current += 1
            max_dd_duration = max(max_dd_duration, current)
        else:
            current = 0

    # === P&L Statistics ===
    gross_profit = df[df['Net Return (%)'] > 0]['Net Return (%)'].sum()
    gross_loss = df[df['Net Return (%)'] < 0]['Net Return (%)'].abs().sum()
    profit_factor = gross_profit / gross_loss if gross_loss != 0 else np.inf

    win_rate = (df['Net Return (%)'] > 0).sum() / (df['Trades'] > 0).sum() * 100
    expectancy = df['Net Return (%)'].mean()

    daily_returns = df['Net Return (%)'].fillna(0)
    sharpe_ratio = daily_returns.mean() / daily_returns.std() * np.sqrt(252) if daily_returns.std() > 0 else 0
    worst_day = daily_returns.min()

    # === Win/Loss Streaks ===
    signs = np.sign(df['Net Return (%)'])
    longest_win = longest_lose = cur_win = cur_lose = 0
    for s in signs:
        if s > 0:
            cur_win += 1
            cur_lose = 0
        elif s < 0:
            cur_lose += 1
            cur_win = 0
        else:
            cur_win = cur_lose = 0
        longest_win = max(longest_win, cur_win)
        longest_lose = max(longest_lose, cur_lose)

    # === Day-wise Return Summary ===
    df['Weekday'] = df.index.day_name()
    weekday_perf = df.groupby('Weekday')['Net Return (%)'].sum().sort_values(ascending=False)

    # === Metrics Return ===
    metrics = {
        'Max Drawdown (%)': max_dd,
        'Max Drawdown Duration (days)': max_dd_duration,
        'Profit Factor': profit_factor,
        'Win Rate (%)': win_rate,
        'Expectancy per Trade (%)': expectancy,
        'Sharpe Ratio': sharpe_ratio,
        'Worst Day Return (%)': worst_day,
        'Longest Win Streak': longest_win,
        'Longest Losing Streak': longest_lose,
        'Weekday Returns': weekday_perf.to_dict()
    }

    return df, metrics

# === Apply the Function ===
df1, metrics = process_strategy(
    strategy1_data,
    strategy1_trades,
    slippage_pct=0.25,
    simulate_missed=True,  # True enables random trade skips
    skip_pct=0.10
)

# === Print Risk Metrics ===
print("\n📊 Strategy Risk Metrics:")
for k, v in metrics.items():
    if isinstance(v, dict):
        print(f"{k}:")
        for wk, val in v.items():
            print(f"   {wk}: {val:.2f}%")
    else:
        print(f"{k}: {v:.2f}")

# === Capital Curve Plot ===
plt.figure(figsize=(20, 8))
plt.plot(df1.index, df1['Capital'], label='Capital', linewidth=2, color='blue')
plt.axhline(100000, color='gray', linestyle='--', label='Initial Capital')
plt.title('Capital Curve')
plt.ylabel('Capital (₹)')
plt.xlabel('Date')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# === Drawdown Curve Plot ===
plt.figure(figsize=(14, 5))
plt.plot(df1.index, df1['Drawdown (%)'], label='Drawdown', linestyle='--', color='red')
plt.axhline(0, color='black', linestyle=':')
plt.title('Drawdown Over Time (%)')
plt.ylabel('Drawdown (%)')
plt.xlabel('Date')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
#Day-wise returns----------------------------------------------

import pandas as pd
import matplotlib.pyplot as plt

strategy1_data = {"2024-11-01": 0.00,
    "2024-11-04": 1.53,"2024-11-05": 10.06,"2024-11-06": 2.14,"2024-11-07": 7.43,"2024-11-08": -2.47,
    "2024-11-11": -1.00,"2024-11-12": -1.76,"2024-11-13": -1.00,"2024-11-14": -1.58,"2024-11-18": -1.72,
    "2024-11-19": -5.13,"2024-11-21": -0.65,"2024-11-22": -3.53,"2024-11-25": -1.00,
    "2024-11-26": -0.98,"2024-11-27": 1.15,"2024-11-28": -0.31, "2024-11-29": 3.98,
    "2024-12-02": 4.50,"2024-12-03": 6.56,"2024-12-04": 1.39,"2024-12-05": 0.83,
    "2024-12-06": 5.70,"2024-12-09": 0.60,"2024-12-10": -0.65,
    "2024-12-11": 0.37,"2024-12-12": -1.73,"2024-12-13": -0.21,"2024-12-16": 2.82,"2024-12-17": 0.00,"2024-12-18": -2.48,
    "2024-12-19": -3.53,"2024-12-20": 0.00,"2024-12-23": 1.26,"2024-12-24": 4.37,"2024-12-26": 4.23,
    "2024-12-27": 4.34,"2024-12-30": 5.20,"2024-12-31": 10.48,

    '2025-01-01': 6.98, '2025-01-02': -5.08, '2025-01-03': 0.17, '2025-01-06': 0.00,
    '2025-01-07': -1.00, '2025-01-08': 7.02, '2025-01-09': -2.71, '2025-01-10': 0.00,
    '2025-01-13': 0.00, '2025-01-14': 8.59, '2025-01-15': -1.00, '2025-01-16': 0.02,
    '2025-01-17': -4.25, '2025-01-20': -4.45, '2025-01-21': 2.70, '2025-01-22': 0.63,
    '2025-01-23': -1.00, '2025-01-24': -3.00, '2025-01-27': 9.79, '2025-01-28': 1.81,
    '2025-01-29': 3.21, '2025-01-30': -4.04, '2025-01-31': 3.25, '2025-02-01': 5.46,
    '2025-02-03': -1.00, '2025-02-04': 5.77, '2025-02-05': -1.64, '2025-02-06': -0.67,
    '2025-02-07': -0.30, '2025-02-10': 2.79, '2025-02-11': 0.00, '2025-02-12': 0.00,
    '2025-02-13': -2.00, '2025-02-14': 0.00, '2025-02-17': 0.00, '2025-02-18': -0.07,
    '2025-02-19': 10.64, '2025-02-20': -1.77, '2025-02-21': 0.00, '2025-02-24': 3.87,
    '2025-02-25': 0.00, '2025-02-27': 10.62, '2025-02-28': -0.02,

    '2025-03-03': 0.00, '2025-03-04': -2.32, '2025-03-05': 0.40, '2025-03-06': -2.20,
    '2025-03-07': -1.36, '2025-03-10': -1.00, '2025-03-11': 6.51, '2025-03-12': -0.27,
    '2025-03-13': 4.03, '2025-03-17': 0.60, '2025-03-18': -0.47, '2025-03-19': 0.18,
    '2025-03-20': 0.53, '2025-03-21': 7.88, '2025-03-24': -1.65, '2025-03-25': 2.39,
    '2025-03-26': 0.00, '2025-03-27': 12.19, '2025-03-28': -3.00,

    '2025-04-01': -3.00, '2025-04-02': -2.24, '2025-04-03': 3.63, '2025-04-04': -1.00,
    '2025-04-07': 1.39, '2025-04-08': -3.33, '2025-04-09': 3.45, '2025-04-11': -1.90,
    '2025-04-15': 6.19, '2025-04-16': -1.37, '2025-04-17': -4.14, '2025-04-21': -0.41,
    '2025-04-22': -1.95, '2025-04-23': 5.33, '2025-04-24': 1.65, '2025-04-25': 2.09,
    '2025-04-28': -0.36, '2025-04-29': 0.38, '2025-04-30': -2.00,
    '2025-05-02': -0.18, '2025-05-05': -0.00, '2025-05-06': 0.00, '2025-05-07': 4.83,
    '2025-05-08': -3.00, '2025-05-09': 6.86, '2025-05-12': 2.09, '2025-05-13': 4.37,
    '2025-05-14': 0.91, '2025-05-15': 6.35, '2025-05-16': 4.18, '2025-05-19': -1.00,
    '2025-05-20': 5.95, '2025-05-21': 2.59, '2025-05-22': 9.65, '2025-05-23': -3.95,
    '2025-05-26': -1.07, '2025-05-27': -2.55, '2025-05-28': 2.58, '2025-05-29': 1.21,
    '2025-05-30': 20.66,

    '2025-06-02': 2.13, '2025-06-03': -2.31, '2025-06-04': 0.19, '2025-06-05': -0.11,
    '2025-06-06': -2.79, '2025-06-09': 1.74, '2025-06-10': 2.04, '2025-06-11': -2.63,
    '2025-06-12': -1.00, '2025-06-13': 3.19, '2025-06-16': 4.96, '2025-06-17': 0.00,
    '2025-06-18': -1.65, '2025-06-19': 0.00, '2025-06-20': -1.86, '2025-06-23': 5.91,
    '2025-06-24': -2.10, '2025-06-25': 4.43, '2025-06-26': 0.61, '2025-06-27': -3.00,
    '2025-06-30': 4.11,

    '2025-07-01': 9.35, '2025-07-02': 0.17, '2025-07-03': 2.73, '2025-07-04': -0.06,
    '2025-07-07': 0.93, '2025-07-08': -1.79, '2025-07-09': -1.96, '2025-07-10': -0.28,
    '2025-07-11': -1.05, '2025-07-14': 1.34
}

strategy1_trades = {
    "2024-11-01": 0,
    "2024-11-04": 2,"2024-11-05": 8,"2024-11-06": 10,"2024-11-07": 3,"2024-11-08": 3,"2024-11-11": 1,
    "2024-11-12": 3,"2024-11-13": 1,"2024-11-14": 2,"2024-11-18": 3,
    "2024-11-19": 6,"2024-11-21": 4,"2024-11-22": 5,"2024-11-25": 1,
    "2024-11-26": 10,"2024-11-27": 8,"2024-11-28": 1,"2024-11-29": 9,

    "2024-12-02": 8,"2024-12-03": 13,"2024-12-04": 1,"2024-12-05": 4,"2024-12-06": 4,
    "2024-12-09": 3,"2024-12-10": 5,"2024-12-11": 10,"2024-12-12": 3,
    "2024-12-13": 2,"2024-12-16": 6,"2024-12-17": 0,"2024-12-18": 6,
    "2024-12-19": 8,"2024-12-20": 0,"2024-12-23": 4,"2024-12-24": 5,
    "2024-12-26": 8,"2024-12-27": 8,"2024-12-30": 3,"2024-12-31": 12,
    '2025-01-01': 10, '2025-01-02': 13, '2025-01-03': 5, '2025-01-06': 0,
    '2025-01-07': 1, '2025-01-08': 4, '2025-01-09': 4, '2025-01-10': 0,
    '2025-01-13': 0, '2025-01-14': 2, '2025-01-15': 1, '2025-01-16': 10,
    '2025-01-17': 7, '2025-01-20': 10, '2025-01-21': 3, '2025-01-22': 1,
    '2025-01-23': 4, '2025-01-24': 3, '2025-01-27': 3, '2025-01-28': 5,
    '2025-01-29': 4, '2025-01-30': 5, '2025-01-31': 6, '2025-02-01': 2,
    '2025-02-03': 1, '2025-02-04': 9, '2025-02-05': 3, '2025-02-06': 6,
    '2025-02-07': 3, '2025-02-10': 2, '2025-02-11': 0, '2025-02-12': 0,
    '2025-02-13': 2, '2025-02-14': 0, '2025-02-17': 0, '2025-02-18': 3,
    '2025-02-19': 2, '2025-02-20': 5, '2025-02-21': 0, '2025-02-24': 3,
    '2025-02-25': 0, '2025-02-27': 6, '2025-02-28': 3,'2025-03-03': 0,
    '2025-03-04': 5, '2025-03-05': 4, '2025-03-06': 4,
    '2025-03-07': 6, '2025-03-10': 1, '2025-03-11': 5, '2025-03-12': 2,
    '2025-03-13': 4, '2025-03-17': 1, '2025-03-18': 12, '2025-03-19': 14,
    '2025-03-20': 3, '2025-03-21': 7, '2025-03-24': 5, '2025-03-25': 1,
    '2025-03-26': 0, '2025-03-27': 16, '2025-03-28': 3,

    '2025-04-01': 3, '2025-04-02': 5, '2025-04-03': 14, '2025-04-04': 1,
    '2025-04-07': 1, '2025-04-08': 5, '2025-04-09': 4, '2025-04-11': 10,
    '2025-04-15': 7, '2025-04-16': 2, '2025-04-17': 15, '2025-04-21': 13,
    '2025-04-22': 8, '2025-04-23': 7, '2025-04-24': 3, '2025-04-25': 1,
    '2025-04-28': 6, '2025-04-29': 3, '2025-04-30': 2,'2025-05-02': 3,
    '2025-05-05': 11, '2025-05-06': 0, '2025-05-07': 4,
    '2025-05-08': 3, '2025-05-09': 7, '2025-05-12': 10, '2025-05-13': 2,
    '2025-05-14': 4, '2025-05-15': 13, '2025-05-16': 11, '2025-05-19': 1,
    '2025-05-20': 3, '2025-05-21': 2, '2025-05-22': 6, '2025-05-23': 8,
    '2025-05-26': 9, '2025-05-27': 4, '2025-05-28': 6, '2025-05-29': 6,
    '2025-05-30': 15,

    '2025-06-02': 8, '2025-06-03': 3, '2025-06-04': 13, '2025-06-05': 11,
    '2025-06-06': 9, '2025-06-09': 12, '2025-06-10': 11, '2025-06-11': 6,
    '2025-06-12': 1, '2025-06-13': 6, '2025-06-16': 9, '2025-06-17': 0,
    '2025-06-18': 3, '2025-06-19': 0, '2025-06-20': 4, '2025-06-23': 12,
    '2025-06-24': 4, '2025-06-25': 6, '2025-06-26': 5, '2025-06-27': 3,
    '2025-06-30': 5,

    '2025-07-01': 6, '2025-07-02': 10, '2025-07-03': 4, '2025-07-04': 2,
    '2025-07-07': 4, '2025-07-08': 5, '2025-07-09': 7, '2025-07-10': 2,
    '2025-07-11': 7, '2025-07-14': 4}

# Step 1: Filter only Monday and Tuesday entries
strategy1_data_filtered = {
    date: ret
    for date, ret in strategy1_data.items()
    if pd.to_datetime(date).weekday() in [0,1,2,3,4]  # Monday=0, Tuesday=1
}

strategy1_trades_filtered = {
    date: trades
    for date, trades in strategy1_trades.items()
    if pd.to_datetime(date).weekday() in [0,1,2,3,4]
}

# Step 2: Process the strategy
def process_monday_tuesday_strategy(data_dict, trades_dict, cost_per_trade=40, initial_capital=100000, leverage=5,
                                    trades_per_day=7, slippage_pct=0.25):
    df = pd.DataFrame({
        'Return (%)': data_dict,
        'Trades': trades_dict
    })
    df.index = pd.to_datetime(df.index)
    df.sort_index(inplace=True)

    df['Adjusted Return (%)'] = df['Return (%)'] * (1 / trades_per_day) * leverage

    slippage_total_pct = slippage_pct / 100
    df['Cost (₹)'] = df['Trades'] * cost_per_trade + (
        initial_capital * slippage_total_pct * (df['Trades'] / trades_per_day)
    )

    df['Net Return (%)'] = df['Adjusted Return (%)'] - (df['Cost (₹)'] / initial_capital * 100)

    df['Capital'] = initial_capital
    for i in range(1, len(df)):
        prev_cap = df.iloc[i - 1]['Capital']
        net_ret = df.iloc[i]['Net Return (%)']
        df.iloc[i, df.columns.get_loc('Capital')] = prev_cap * (1 + net_ret / 100)

    return df

# Process only Monday and Tuesday trades
df_filtered = process_monday_tuesday_strategy(strategy1_data_filtered, strategy1_trades_filtered)

# Step 3: Plot the capital curve
plt.figure(figsize=(18, 6))
plt.plot(df_filtered.index, df_filtered['Capital'], marker='o', linewidth=2, color='green')
plt.axhline(100000, linestyle='--', color='gray', label='Initial Capital (₹1,00,000)')
plt.title("Capital Growth Curve (Only Mondays & Tuesdays)")
plt.xlabel("Date")
plt.ylabel("Capital (₹)")
plt.grid(True)
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

# Final capital
final_capital = df_filtered['Capital'].iloc[-1]
print(f"\n📈 Final Capital (Only Monday & Tuesday Trading): ₹{final_capital:,.2f}")